In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_openai.chat_models import ChatOpenAI
from typing import TypedDict,Annotated
from langgraph.graph.message import add_messages
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import ToolNode,tools_condition
from langchain_core.tools import tool
import sqlite3
import requests
from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import BaseMessage, HumanMessage

In [2]:
#activate environment
load_dotenv()

True

In [3]:
#llm call
llm=ChatOpenAI()

In [4]:
#tool 1 duckduckgo
search_tool= DuckDuckGoSearchRun(region="us-en")

In [5]:
#tool 2 duckduckgo
@tool
def calculater(input1 : float ,input2 : float ,operation : str) -> dict:
    """This is a calculator tool. This is used to perform operation such as add , subtract , 
       multiply and divide based on operation mentioned between input1 and input2
    """
    try:
        if operation =="add":
            result = input1 + input2
        elif operation =="subtract":
            result = input1 - input2
        elif operation =="multiply":
            result = input1 * input2
        elif operation =="divide":
            result = input1 / input2
        else:
            return {"error": f"Unsupported operation '{operation}'"}
        
        return {"input1": input1, "input2": input2, "operation": operation, "result": result}
    except Exception as e:
        return {"error": str(e)}



In [6]:
#tool 3 ,search of stocks prices
def get_stock_price(symbol : str)->dict:
        """
          Fetch latest stock price for a given symbol (e.g. 'AAPL', 'TSLA') 
          using Alpha Vantage with API key in the URL.
        """
        url=f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey=8YKKJTTR02SRUN6K'
        r=requests.get(url)

        return r.json()

In [7]:
#bind tools
tools_list=[search_tool,calculater,get_stock_price]
llm_with_tools=llm.bind_tools(tools_list)

In [8]:
#create class ChatState
class ChatState(TypedDict):
    messages:Annotated[list[BaseMessage],add_messages]


In [9]:
#Node
def ChatNode(state:ChatState):
    """Node that may answer the user query or request for a tool"""
    messages=state['messages']
    response=llm_with_tools.invoke(messages)
    return {'messages':[response]}

In [10]:
#tool node
tools=ToolNode(tools_list)

In [11]:
#checkpointer
conn=sqlite3.connect(database='chatbot.db',check_same_thread=False)
checkpointer=SqliteSaver(conn=conn)

In [12]:
#graph building
graph=StateGraph(ChatState)

graph.add_node("ChatNode",ChatNode)

graph.add_node("tools",tools)


In [13]:
graph.add_edge(START,"ChatNode")

graph.add_conditional_edges("ChatNode",tools_condition)

graph.add_edge("tools","ChatNode")

#graph.add_edge("ChatNode",END)



In [14]:
#chatbot=graph.compile(checkpointer=checkpointer)
chatbot=graph.compile()

In [15]:
output=chatbot.invoke({'messages':[HumanMessage(content="what is 21 divide by 21 " 
                                                        )]})
output['messages'][-1].content

'The result of dividing 21 by 21 is 1.'

In [20]:
output=chatbot.invoke({'messages':[HumanMessage(content="What is the stock price of Microsoft. What is the price of 10 stocks " 
                                                        )]})
output['messages'][-1].content

'- The current stock price of Microsoft (MSFT) is $507.23.\n- The price of 10 stocks of Microsoft would be $5072.30.'